In [1]:
import numpy as np
import math
import torch
from torch import nn

import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt

from tempfile import TemporaryDirectory

from Models import BigGenerator
from Tools import KL, batchKL, sw, FunSW, FunKL

from tqdm import trange

from Experiments import get_setup

import itertools

from pandas import DataFrame
import pandas as pd

print(torch.__version__)


1.4.0


In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [24]:
lat_dim=5

In [25]:
models=torch.load('Results/models_es.pt')
pd.DataFrame(models)

,foong,foong_sparse,boston,concrete,energy,powerplant,wine,yacht
HMC,"[[[tensor(-0.1906), tensor(-0.6890), tensor(-0...","[[[tensor(0.0301), tensor(1.2959), tensor(0.37...","[[[tensor(1.0707), tensor(0.0194), tensor(1.54...","[[[tensor(0.5101), tensor(-0.2090), tensor(0.8...","[[[tensor(-0.2692), tensor(1.1072), tensor(0.7...","[[[tensor(-1.2375), tensor(-1.1718), tensor(-0...","[[[tensor(-0.3964), tensor(-0.9182), tensor(-1...","[[[tensor(-0.3975), tensor(0.6449), tensor(-0...."
GeNNeVI,"{'0': {'model.0.weight': [tensor([-0.5057, -0....","{'0': {'model.0.weight': [tensor([-0.2891, 0....","{'0': {'model.0.weight': [tensor([ 0.7990, 1....","{'0': {'model.0.weight': [tensor([-0.2699, -0....","{'0': {'model.0.weight': [tensor([-0.2387, -0....","{'0': {'model.0.weight': [tensor([ 0.1301, -0....","{'0': {'model.0.weight': [tensor([-0.4440, 0....","{'0': {'model.0.weight': [tensor([ 0.2615, 0...."
FuNNeVI,"{'0': {'model.0.weight': [tensor([-0.0067, 0....","{'0': {'model.0.weight': [tensor([ 0.5846, 0....","{'0': {'model.0.weight': [tensor([ 0.0952, -0....","{'0': {'model.0.weight': [tensor([ 0.7364, -0....","{'0': {'model.0.weight': [tensor([-0.4484, 0....","{'0': {'model.0.weight': [tensor([ 0.1503, 0....","{'0': {'model.0.weight': [tensor([-0.0226, 0....","{'0': {'model.0.weight': [tensor([ 0.0287, -0...."


In [15]:
datasets=[d for d,i in models.items()]
datasets

['foong',
 'foong_sparse',
 'boston',
 'concrete',
 'energy',
 'powerplant',
 'wine',
 'yacht']

In [16]:
results_f=torch.load('Results/results_f.pt')


In [17]:
metrics=[m for m,d in results_f.items()]
metrics

['KL(G,HMC)',
 'KL(F,HMC)',
 'KL(HMC,G)',
 'KL(HMC,F)',
 'SW(HMC,G)',
 'SW(HMC,F)',
 'KL(G,G)',
 'KL(F,F)',
 'SW(G,G)',
 'SW(F,F)']

In [18]:
pd.DataFrame(results_f)

,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(93.609, 3.109)","(79.702, 3.106)","(93.524, 4.952)","(87.388, 3.365)","(0.056, 0.001)","(0.154, 0.017)","(48.479, 17.907)","(24.46, 2.381)",None,None
foong_sparse,"(74.764, 3.025)","(78.206, 16.89)","(40.717, 4.308)","(57.645, 12.36)","(0.019, 0.002)","(0.048, 0.016)","(39.101, 3.057)","(46.621, 18.303)",None,None
boston,"(66.876, 2.614)","(9.188, 0.924)","(9.968, 0.375)","(52.984, 1.533)","(0.427, 0.002)","(1.562, 0.021)","(12.405, 3.185)","(8.26, 1.256)",None,None
concrete,"(61.895, 1.988)","(9.743, 1.341)","(5.907, 0.21)","(49.093, 1.469)","(0.315, 0.004)","(1.008, 0.011)","(3.585, 0.841)","(8.621, 1.571)",None,None
energy,"(44.847, 12.241)","(10.915, 3.191)","(12.987, 3.607)","(51.945, 2.846)","(1.077, 1.059)","(0.661, 0.018)","(34.873, 9.787)","(8.889, 4.076)",None,None
powerplant,"(99.755, 1.941)","(16.784, 1.722)","(9.343, 0.337)","(74.657, 1.99)","(0.108, 0.0)","(0.741, 0.013)","(26.304, 13.344)","(14.214, 2.599)",None,None
wine,"(79.141, 5.465)","(78.098, 7.11)","(20.131, 3.774)","(37.968, 8.555)","(0.623, 0.077)","(1.119, 0.216)","(63.66, 5.092)","(78.027, 9.719)",None,None
yacht,"(97.657, 4.306)","(11.403, 1.586)","(14.405, 1.089)","(55.081, 1.802)","(0.238, 0.001)","(0.625, 0.006)","(13.611, 1.388)","(7.562, 2.14)",None,None


In [19]:
ml_path='mlruns/'

In [20]:
models['boston']['FuNNeVI']=torch.load(ml_path+'21/0848470a544c4c6182c73da2bff2780d/artifacts/models.pt') #

models['yacht']['FuNNeVI']=torch.load(ml_path+'22/3fb22e52fe3b4529b330d80a03fcdc6d/artifacts/models.pt')

models['concrete']['FuNNeVI']=torch.load(ml_path+'23/1f9801862dc04cb4bb9966d663b02958/artifacts/models.pt')

models['energy']['FuNNeVI']=torch.load(ml_path+'24/b67d98b045f347fe9f6f6a91e5fec8be/artifacts/models.pt')


models['wine']['FuNNeVI']=torch.load(ml_path+'25/c22377ca4242476fa755803e4153237d/artifacts/models.pt')

models['powerplant']['FuNNeVI']=torch.load(ml_path+'26/1aa6af0772aa4aaeaa8498e993cef29e/artifacts/models.pt')

models['foong']['FuNNeVI']=torch.load(ml_path+'28/4a2ae72d67d945c0bc7b3df345a93321/artifacts/models.pt')

#models['foong_mixed']['FuNNeVI']=torch.load(ml_path+'17/07025622b9594b62a8bdf231d1c7957c/artifacts/models.pt')
#models['foong_mixed']['GeNNeVI']=torch.load(ml_path+'18/0a0c7376df1541ada31a795d072bf738/artifacts/models.pt')

models['foong_sparse']['FuNNeVI']=torch.load(ml_path+'30/117a6bb3054444d6b12c65618a36bdf5/artifacts/models.pt')


In [26]:




torch.save(models, 'Results/models_lp.pt')



In [27]:
metric='KL(G,HMC)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(s,t,projection,device)
        print(K)
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


tensor(106.3064)
tensor(157.8185)
tensor(86.5107)
[106.30644226074219, 157.81849670410156, 86.51068115234375]
tensor(51.8193)
tensor(65.9424)
tensor(73.5312)
[51.81926345825195, 65.94237518310547, 73.53121948242188]
tensor(66.5829)
tensor(68.4329)
tensor(64.1689)
[66.58293151855469, 68.43286895751953, 64.16886901855469]
tensor(66.7690)
tensor(65.2771)
tensor(62.4959)
[66.76895141601562, 65.27713775634766, 62.49593734741211]
tensor(36.0470)
tensor(48.8263)
tensor(39.2022)
[36.046958923339844, 48.8262825012207, 39.20217514038086]
tensor(88.2925)
tensor(88.9494)
tensor(92.2796)
[88.29251098632812, 88.94935607910156, 92.27961730957031]
tensor(88.2539)
tensor(83.4378)
tensor(88.8902)
[88.25394439697266, 83.4378433227539, 88.89021301269531]
tensor(100.0440)
tensor(102.1373)
tensor(102.0101)
[100.04400634765625, 102.13729858398438, 102.01014709472656]


In [28]:
metric='KL(F,HMC)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(s,t,projection,device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 83.29922485351562
foong: 84.33883666992188
foong: 73.63076782226562
[83.29922485351562, 84.33883666992188, 73.63076782226562]
foong_sparse: 90.34156036376953
foong_sparse: 74.27351379394531
foong_sparse: 78.8741683959961
[90.34156036376953, 74.27351379394531, 78.8741683959961]
boston: 10.579803466796875
boston: 9.300848007202148
boston: 12.303353309631348
[10.579803466796875, 9.300848007202148, 12.303353309631348]
concrete: 13.059745788574219
concrete: 11.335906028747559
concrete: 11.600337028503418
[13.059745788574219, 11.335906028747559, 11.600337028503418]
energy: 13.311158180236816
energy: 12.921976089477539
energy: 14.062399864196777
[13.311158180236816, 12.921976089477539, 14.062399864196777]
powerplant: 18.8214054107666
powerplant: 18.61473846435547
powerplant: 17.873491287231445
[18.8214054107666, 18.61473846435547, 17.873491287231445]
wine: 84.72220611572266
wine: 93.00473022460938
wine: 72.63961791992188
[84.72220611572266, 93.00473022460938, 72.63961791992188]
yacht: 

In [29]:
metric='KL(HMC,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(t,s,projection,device)
        if dataset == 'powerplant' or dataset == 'wine':
                K=FunKL(t,s,projection,device,k=2)

        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 99.97187805175781
foong: 127.44656372070312
foong: 87.3095703125
[99.97187805175781, 127.44656372070312, 87.3095703125]
foong_sparse: 32.12327575683594
foong_sparse: 42.664485931396484
foong_sparse: 44.241065979003906
[32.12327575683594, 42.664485931396484, 44.241065979003906]
boston: 9.295921325683594
boston: 8.793379783630371
boston: 9.773567199707031
[9.295921325683594, 8.793379783630371, 9.773567199707031]
concrete: 6.239816188812256
concrete: 6.5078630447387695
concrete: 6.006532669067383
[6.239816188812256, 6.5078630447387695, 6.006532669067383]
energy: 11.795995712280273
energy: 28.535411834716797
energy: 5.243054389953613
[11.795995712280273, 28.535411834716797, 5.243054389953613]
powerplant: 7.981212615966797
powerplant: 8.225085258483887
powerplant: 7.151486396789551
[7.981212615966797, 8.225085258483887, 7.151486396789551]
wine: 24.150278091430664
wine: 22.157915115356445
wine: 23.583616256713867
[24.150278091430664, 22.157915115356445, 23.583616256713867]
yacht: 14.8

In [30]:
metric='KL(HMC,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)

    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunKL(t,s,projection,device)
        if dataset == 'powerplant' or dataset == 'wine':
            K=FunKL(t,s,projection,device,k=2)

        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: 64.50849914550781
foong: 46.6417350769043
foong: 108.8575210571289
[64.50849914550781, 46.6417350769043, 108.8575210571289]
foong_sparse: 62.768959045410156
foong_sparse: 52.247615814208984
foong_sparse: 53.44927215576172
[62.768959045410156, 52.247615814208984, 53.44927215576172]
boston: 52.011207580566406
boston: 53.866764068603516
boston: 51.25685119628906
[52.011207580566406, 53.866764068603516, 51.25685119628906]
concrete: 45.586280822753906
concrete: 47.081016540527344
concrete: 44.202030181884766
[45.586280822753906, 47.081016540527344, 44.202030181884766]
energy: 48.39130783081055
energy: 49.671470642089844
energy: 47.99309158325195
[48.39130783081055, 49.671470642089844, 47.99309158325195]
powerplant: 70.94680786132812
powerplant: 71.78516387939453
powerplant: 69.4460678100586
[70.94680786132812, 71.78516387939453, 69.4460678100586]
wine: 31.012775421142578
wine: 32.11348342895508
wine: 40.74114990234375
[31.012775421142578, 32.11348342895508, 40.74114990234375]
yacht: 

In [31]:
metric='SW(HMC,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunSW(t,s, projection, device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))



foong: 0.06704457849264145
foong: 0.09644351154565811
foong: 0.04333372414112091
[0.06704457849264145, 0.09644351154565811, 0.04333372414112091]
foong_sparse: 0.015064842998981476
foong_sparse: 0.01439884677529335
foong_sparse: 0.020075209438800812
[0.015064842998981476, 0.01439884677529335, 0.020075209438800812]
boston: 0.42760494351387024
boston: 0.42318451404571533
boston: 0.4234927296638489
[0.42760494351387024, 0.42318451404571533, 0.4234927296638489]
concrete: 0.3174227178096771
concrete: 0.320745587348938
concrete: 0.3178612291812897
[0.3174227178096771, 0.320745587348938, 0.3178612291812897]
energy: 0.3543526530265808
energy: 0.5649887323379517
energy: 0.24985100328922272
[0.3543526530265808, 0.5649887323379517, 0.24985100328922272]
powerplant: 0.10277524590492249
powerplant: 0.10418353974819183
powerplant: 0.10543803125619888
[0.10277524590492249, 0.10418353974819183, 0.10543803125619888]
wine: 0.6962540149688721
wine: 0.7489683628082275
wine: 0.7192153334617615
[0.69625401496

In [32]:
metric='SW(HMC,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    KLs=[]
    for i,m in models[dataset][method].items():
        t=models[dataset]['HMC'][0].to(device)
        G.load_state_dict(m)
        s=G(t.shape[0]).detach()
        K=FunSW(t,s, projection, device)
        print(dataset+': '+str(K.item()))
        KLs.append(K.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))



foong: 0.10549516975879669
foong: 0.10283024609088898
foong: 0.17109224200248718
[0.10549516975879669, 0.10283024609088898, 0.17109224200248718]
foong_sparse: 0.0726858377456665
foong_sparse: 0.0282409880310297
foong_sparse: 0.0398910753428936
[0.0726858377456665, 0.0282409880310297, 0.0398910753428936]
boston: 1.5777461528778076
boston: 1.5828078985214233
boston: 1.5925642251968384
[1.5777461528778076, 1.5828078985214233, 1.5925642251968384]
concrete: 1.0066545009613037
concrete: 1.0307765007019043
concrete: 0.9644888043403625
[1.0066545009613037, 1.0307765007019043, 0.9644888043403625]
energy: 0.6497585773468018
energy: 0.6499261260032654
energy: 0.6543932557106018
[0.6497585773468018, 0.6499261260032654, 0.6543932557106018]
powerplant: 0.6715003848075867
powerplant: 0.7076734900474548
powerplant: 0.7268797159194946
[0.6715003848075867, 0.7076734900474548, 0.7268797159194946]
wine: 0.8323590755462646
wine: 1.18373441696167
wine: 1.895551323890686
[0.8323590755462646, 1.18373441696167

In [33]:
metric='KL(G,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    KLs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        K=FunKL(t,s, projection,device)
        K_=FunKL(s,t, projection,device)
        print(dataset+': '+str((K.item(), K_.item())))
        KLs.append(K.item())
        KLs.append(K_.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: (88.47074127197266, 98.49993133544922)
foong: (123.10397338867188, 116.27713012695312)
foong: (172.7467803955078, 162.29354858398438)
[88.47074127197266, 98.49993133544922, 123.10397338867188, 116.27713012695312, 172.7467803955078, 162.29354858398438]
foong_sparse: (57.8257942199707, 41.11382293701172)
foong_sparse: (64.63041687011719, 35.06955337524414)
foong_sparse: (55.36235427856445, 43.91758346557617)
[57.8257942199707, 41.11382293701172, 64.63041687011719, 35.06955337524414, 55.36235427856445, 43.91758346557617]
boston: (12.305706024169922, 9.025298118591309)
boston: (10.007223129272461, 11.329902648925781)
boston: (9.448931694030762, 13.285820007324219)
[12.305706024169922, 9.025298118591309, 10.007223129272461, 11.329902648925781, 9.448931694030762, 13.285820007324219]
concrete: (5.7931227684021, 6.353964328765869)
concrete: (4.897809028625488, 7.228177070617676)
concrete: (3.133174419403076, 5.534015655517578)
[5.7931227684021, 6.353964328765869, 4.897809028625488, 7.22

In [34]:
metric='KL(F,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))
    
    def projection(t,s,m):
        return setup.projection(t,s,m,ratio_ood=1)
    
    
    KLs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        K=FunKL(t,s, projection,device)
        K_=FunKL(s,t, projection,device)
        print(dataset+': '+str((K.item(), K_.item())))
        KLs.append(K.item())
        KLs.append(K_.item())
    print(KLs)
    results_f[metric][dataset]=(np.mean(KLs).round(decimals=3), np.std(KLs).round(decimals=3))


foong: (52.1048469543457, 48.3261604309082)
foong: (50.251060485839844, 64.58181762695312)
foong: (51.0212516784668, 87.49661254882812)
[52.1048469543457, 48.3261604309082, 50.251060485839844, 64.58181762695312, 51.0212516784668, 87.49661254882812]
foong_sparse: (45.17316818237305, 64.53996276855469)
foong_sparse: (33.434635162353516, 56.00629806518555)
foong_sparse: (40.30446243286133, 33.367897033691406)
[45.17316818237305, 64.53996276855469, 33.434635162353516, 56.00629806518555, 40.30446243286133, 33.367897033691406]
boston: (7.93752908706665, 9.836756706237793)
boston: (10.85765266418457, 8.977888107299805)
boston: (11.461533546447754, 7.612913608551025)
[7.93752908706665, 9.836756706237793, 10.85765266418457, 8.977888107299805, 11.461533546447754, 7.612913608551025]
concrete: (8.57712173461914, 11.102191925048828)
concrete: (10.152570724487305, 10.209875106811523)
concrete: (11.0693998336792, 8.279532432556152)
[8.57712173461914, 11.102191925048828, 10.152570724487305, 10.2098751

In [35]:
metric='SW(G,G)'
for dataset in datasets:
    method='GeNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    SWs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        SW=sw(s.cpu(),t.cpu(),'cpu')
        print(SW)
        SWs.append(SW.item())
    
    results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))


1.0139061218181662
0.9678144342822522
1.0360357091969015


NameError: name 'results_p' is not defined

In [ ]:
metric='SW(F,F)'
for dataset in datasets:
    method='FuNNeVI'

    setup_ = get_setup(dataset)
    setup=setup_.Setup(device) 
    G=BigGenerator(lat_dim, setup.param_count, device).to(device)
    models_pairs=list(itertools.combinations(models[dataset][method].items(),2))

    SWs=[]
    for (i,m),(j,n) in models_pairs:
        G.load_state_dict(m)
        s=G(10000).detach()
        G.load_state_dict(n)
        t=G(10000).detach()
        SW=sw(s.cpu(),t.cpu(),'cpu')
        print(SW)
        SWs.append(SW.item())
    
    results_f[metric][dataset]=(np.mean(SWs).round(decimals=3), np.std(SWs).round(decimals=3))


In [36]:
torch.save(results_f, 'Results/results_f_lp.pt')
pd.DataFrame(results_f)


,"KL(G,HMC)","KL(F,HMC)","KL(HMC,G)","KL(HMC,F)","SW(HMC,G)","SW(HMC,F)","KL(G,G)","KL(F,F)","SW(G,G)","SW(F,F)"
foong,"(116.879, 30.056)","(80.423, 4.822)","(104.909, 16.754)","(73.336, 26.155)","(0.069, 0.022)","(0.126, 0.032)","(126.899, 30.998)","(58.964, 13.814)",None,None
foong_sparse,"(63.764, 8.997)","(81.163, 6.756)","(39.676, 5.379)","(56.155, 4.702)","(0.017, 0.003)","(0.047, 0.019)","(49.653, 10.346)","(45.471, 11.495)",None,None
boston,"(66.395, 1.746)","(10.728, 1.23)","(9.288, 0.4)","(52.378, 1.097)","(0.425, 0.002)","(1.584, 0.006)","(10.9, 1.542)","(9.447, 1.418)",None,None
concrete,"(64.847, 1.771)","(11.999, 0.758)","(6.251, 0.205)","(45.623, 1.176)","(0.319, 0.001)","(1.001, 0.027)","(5.49, 1.276)","(9.898, 1.107)",None,None
energy,"(41.358, 5.435)","(13.432, 0.473)","(15.191, 9.807)","(48.685, 0.716)","(0.39, 0.131)","(0.651, 0.002)","(36.627, 13.077)","(10.245, 0.56)",None,None
powerplant,"(89.84, 1.745)","(18.437, 0.407)","(7.786, 0.46)","(70.726, 0.968)","(0.104, 0.001)","(0.702, 0.023)","(28.06, 10.61)","(16.169, 0.768)",None,None
wine,"(86.861, 2.434)","(83.456, 8.362)","(23.297, 0.838)","(34.622, 4.35)","(0.721, 0.022)","(1.304, 0.442)","(76.495, 2.576)","(83.15, 14.328)",None,None
yacht,"(101.397, 0.958)","(13.727, 2.47)","(14.151, 0.531)","(52.081, 1.302)","(0.245, 0.008)","(0.607, 0.004)","(26.186, 10.898)","(9.388, 2.836)",None,None
